In [24]:
import pandas as pd
import numpy as np
import wrangle
import matplotlib.pyplot as plt
import seaborn as sns

# modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import TweedieRegressor

import acquire
import prepare
import wrangle

import warnings
warnings.filterwarnings("ignore")

In [25]:
train, validate, test, X_train, y_train, X_validate, y_validate, X_test, y_test, scaler, X_train_scaled, X_validate_scaled, X_test_scaled = wrangle.wrangle_zillow(target = "tax_value")

In [26]:
df = acquire.create_zillow_data()

In [27]:
df.head()

,parcelid,id,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,logerror,transactiondate
0,14634203,2026522,1.0,NaN,NaN,2.0,3.0,None,NaN,2.0,...,289445.0,2016.0,213516.0,3490.16,None,NaN,6.059001e+13,625,-0.061973,2017-08-04
1,11721753,616260,NaN,NaN,NaN,2.0,3.0,None,6.0,2.0,...,205123.0,2016.0,163175.0,2627.48,None,NaN,6.037220e+13,1018,0.017785,2017-07-21
2,11289917,2061546,1.0,NaN,NaN,2.0,3.0,None,6.0,2.0,...,136104.0,2016.0,27214.0,2319.90,Y,15.0,6.037901e+13,1248,-0.362001,2017-06-23
3,11637029,2554497,1.0,NaN,NaN,2.0,3.0,None,9.0,2.0,...,810694.0,2016.0,554156.0,9407.01,Y,13.0,6.037701e+13,1734,-0.093399,2017-07-28
4,11705026,1834372,NaN,NaN,NaN,1.0,2.0,None,6.0,1.0,...,35606.0,2016.0,23624.0,543.69,None,NaN,6.037232e+13,1772,-0.146056,2017-06-30


In [28]:
df.columns

Index(['parcelid', 'id', 'airconditioningtypeid', 'architecturalstyletypeid',
       'basementsqft', 'bathroomcnt', 'bedroomcnt', 'buildingclasstypeid',
       'buildingqualitytypeid', 'calculatedbathnbr', 'decktypeid',
       'finishedfloor1squarefeet', 'calculatedfinishedsquarefeet',
       'finishedsquarefeet12', 'finishedsquarefeet13', 'finishedsquarefeet15',
       'finishedsquarefeet50', 'finishedsquarefeet6', 'fips', 'fireplacecnt',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'storytypeid', 'threequarterbathnbr', 'typeconstructiontypeid',
       'unitcnt', 'yardbuildingsqft17', 'yardb

In [29]:
df.regionidcounty.unique()

array([1286., 3101., 2061.])

In [30]:
df.regionidcity.unique()

array([ 18874.,  12447.,   5534.,  10389.,  38032.,  54722., 396550.,
        45398.,  33837.,  46098.,  41673.,  30908.,  33252.,  37086.,
        32380.,  24832.,  46298.,  44116.,  14542.,  16764.,  45602.,
        47547.,  37015.,  45457.,  20008.,  40227., 396053.,  26964.,
        47019., 118217.,  34543., 396551.,  24174.,  54311.,  34780.,
         5465.,  54212.,  10608.,  17686.,  17882.,  13150.,  53027.,
       113576.,  48424., 396054.,  50749., 118895.,  25218.,  27491.,
        27110.,  19177.,  52650.,  47568.,  12292.,  17150.,     nan,
        24812.,  12773.,  29712.,  22827.,  53571.,  51239.,  44833.,
       118225., 118878.,   4406.,  14634.,  51861.,  21412.,  53636.,
       118994.,  13091.,  37688.,  34278.,  39306.,  46080.,  50677.,
        13693.,  25953., 114834.,  40081.,  39308.,  25459.,  13716.,
        14111.,  47198.,  12520.,  54352.,  25458.,  10774.,  52842.,
        47762., 118875.,  33836.,  11626.,  24245.,  42150.,   8384.,
        26483.,  329

In [31]:
tax_info = acquire.zip_code_zillow(df)

In [34]:
tax_info.head()

,parcelid,tax_value,fips,tax_amount,longitude,latitude
0,14634203,289445.0,6059.0,3490.16,-117954970.0,33922098.0
1,11721753,205123.0,6037.0,2627.48,-118355396.0,34033180.0
2,11289917,136104.0,6037.0,2319.90,-118113100.0,34686163.0
3,11637029,810694.0,6037.0,9407.01,-118390965.0,34073222.0
4,11705026,35606.0,6037.0,543.69,-118291863.0,33999877.0


In [37]:
tax_info["tax_rate"] = tax_info.tax_amount / tax_info.tax_value

In [38]:
tax_info

,parcelid,tax_value,fips,tax_amount,longitude,latitude,tax_rate
0,14634203,289445.0,6059.0,3490.16,-117954970.0,33922098.0,0.012058
1,11721753,205123.0,6037.0,2627.48,-118355396.0,34033180.0,0.012809
2,11289917,136104.0,6037.0,2319.90,-118113100.0,34686163.0,0.017045
3,11637029,810694.0,6037.0,9407.01,-118390965.0,34073222.0,0.011604
4,11705026,35606.0,6037.0,543.69,-118291863.0,33999877.0,0.015270
...,...,...,...,...,...,...,...
38614,14453399,852813.0,6059.0,11662.88,-117578817.0,33468267.0,0.013676
38615,12019061,492395.0,6037.0,6023.37,-118377000.0,34092100.0,0.012233
38616,11570288,600608.0,6037.0,7255.87,-118471000.0,34050300.0,0.012081
38617,11128688,1120000.0,6037.0,13663.03,-118554206.0,34279343.0,0.012199


In [39]:
tax_info.fips.unique()

array([6059., 6037., 6111.])

In [41]:
tax_info["county"] = tax_info["fips"].map({6037:"Los Angeles", 6059:"Orange", 6111:"Ventura"})

In [42]:
tax_info.head()

,parcelid,tax_value,fips,tax_amount,longitude,latitude,tax_rate,county
0,14634203,289445.0,6059.0,3490.16,-117954970.0,33922098.0,0.012058,Orange
1,11721753,205123.0,6037.0,2627.48,-118355396.0,34033180.0,0.012809,Los Angeles
2,11289917,136104.0,6037.0,2319.90,-118113100.0,34686163.0,0.017045,Los Angeles
3,11637029,810694.0,6037.0,9407.01,-118390965.0,34073222.0,0.011604,Los Angeles
4,11705026,35606.0,6037.0,543.69,-118291863.0,33999877.0,0.015270,Los Angeles


In [46]:
county_average_tax_rate = pd.DataFrame(round(tax_info.groupby("county").tax_rate.mean(),5)*100)
county_average_tax_rate.columns = ["tax_rate_percent"]
county_average_tax_rate

,tax_rate_percent
county,
Los Angeles,1.374
Orange,1.204
Ventura,1.188
